In [1]:
import pandas as pd
import numpy as np

# Validation

In [2]:
train_data = pd.read_csv('in.dta',header=None,names=['x1','x2','y'])
test_data = pd.read_csv('out.dta',header=None,names=['x1','x2','y'])

In [3]:
def split_data(data,k):
    column_names = ['1','x1','x2']
    data['1'] = 1
    if k >= 3:
        data['x1^2']=data['x1']*data['x1']
        column_names.append('x1^2')
    if k >= 4:
        data['x2^2']=data['x2']*data['x2']
        column_names.append('x2^2')
    if k >= 5:
        data['x1x2']=data['x1']*data['x2']
        column_names.append('x1x2')
    if k >= 6:
        data['|x1-x2|']=abs(data['x1']-data['x2'])
        column_names.append('|x1-x2|')
    if k >= 7:
        data['|x1+x2|']=abs(data['x1']+data['x2'])
        column_names.append('|x1+x2|')
    H = pd.DataFrame(data,columns=column_names).values
    label = data['y'].values
    return H,label

In [4]:
def split_train_val(data,k):
    column_names = ['1','x1','x2']
    data['1'] = 1
    if k >= 3:
        data['x1^2']=data['x1']*data['x1']
        column_names.append('x1^2')
    if k >= 4:
        data['x2^2']=data['x2']*data['x2']
        column_names.append('x2^2')
    if k >= 5:
        data['x1x2']=data['x1']*data['x2']
        column_names.append('x1x2')
    if k >= 6:
        data['|x1-x2|']=abs(data['x1']-data['x2'])
        column_names.append('|x1-x2|')
    if k >= 7:
        data['|x1+x2|']=abs(data['x1']+data['x2'])
        column_names.append('|x1+x2|')
    H = pd.DataFrame(data,columns=column_names).values
    label = data['y'].values
    trainH = H[:25]
    trainlabel = label[:25]
    valH = H[25:]
    vallabel = label[25:]
    return trainH,trainlabel,valH,vallabel

In [5]:
def slove_LRegress(X,y):
    temp = np.dot(X.T,X)
    pseudo_inverse = np.dot(np.linalg.inv(temp),X.T)
    wlin = np.dot(pseudo_inverse,y)
    return wlin

In [6]:
def compute_Error(H,label,w):
    N = H.shape[0]
    predicted = np.ones(N)
    predicted[np.dot(H,w)<0]=-1
    errors = (predicted!=label).sum()
    return errors*1.0/N

In [7]:
train_H,train_label,val_H,val_labe = split_train_val(train_data,3)
#test_H,test_label = split_data(test_data)

## Question 1

In [8]:
val_error = 1
small_k=3
for k in range(3,8):
    train_H,train_label,val_H,val_label = split_train_val(train_data,k)
    wlin = slove_LRegress(train_H,train_label)
    error = compute_Error(val_H,val_label,wlin)
    if error < val_error:
        val_error = error
        small_k = k
    print 'E_val',k,error
print 'smallest_k',small_k

E_val 3 0.3
E_val 4 0.5
E_val 5 0.2
E_val 6 0.0
E_val 7 0.1
smallest_k 6


## Question 2

In [9]:
val_error = 1
small_k=3
for k in range(3,8):
    train_H,train_label,val_H,val_label = split_train_val(train_data,k)
    #train_H,train_label = split_data(train_data,k)
    test_H,test_label = split_data(test_data,k)
    wlin = slove_LRegress(train_H,train_label)
    error = compute_Error(test_H,test_label,wlin)
    if error < val_error:
        val_error = error
        small_k = k
    print 'E_val',k,error
print 'smallest_k',small_k

E_val 3 0.42
E_val 4 0.416
E_val 5 0.188
E_val 6 0.084
E_val 7 0.072
smallest_k 7


## Question 3

In [10]:
val_error = 1
small_k=3
for k in range(3,8):
    train_H,train_label,val_H,val_label = split_train_val(train_data,k)
    wlin = slove_LRegress(val_H,val_label)
    error = compute_Error(train_H,train_label,wlin)
    if error < val_error:
        val_error = error
        small_k = k
    print 'E_val',k,error
print 'smallest_k',small_k

E_val 3 0.28
E_val 4 0.36
E_val 5 0.2
E_val 6 0.08
E_val 7 0.12
smallest_k 6


## Question 4

In [11]:
val_error = 1
small_k=3
for k in range(3,8):
    train_H,train_label,val_H,val_label = split_train_val(train_data,k)
    test_H,test_label = split_data(test_data,k)
    wlin = slove_LRegress(val_H,val_label)
    error = compute_Error(test_H,test_label,wlin)
    if error < val_error:
        val_error = error
        small_k = k
    print 'E_val',k,error
print 'smallest_k',small_k

E_val 3 0.396
E_val 4 0.388
E_val 5 0.284
E_val 6 0.192
E_val 7 0.196
smallest_k 6


# PLA vs. SVM

In [12]:
def sign(x,w):
    """
    x should be a np.array of shape(3L,)
    """
    inproduct = np.dot(w,x) 
    if inproduct > 0:
        return 1
    elif inproduct < 0:
        return -1
    else:
        return 0

In [13]:
def checkmistakes(data,label,WPLA):
    numofdata = data.shape[0]
    for i in range(numofdata):
        if sign(data[i],WPLA) != label[i]:
            return False
    return True

In [14]:
def PLA(data,label,WPLA):    
    #初始化WPLA
    numofdata = data.shape[0]
    t = 0
    while True:
        for i in range(numofdata):
            if sign(data[i],WPLA) != label[i]:
                t+=1
                WPLA += label[i]*data[i]
        if checkmistakes(data,label,WPLA):
            return WPLA

In [15]:
def line(x,w):
    return (-w[0]-w[1]*x)/w[2]

In [16]:
def exp(N):
    data = np.random.uniform(-1, 1, (N, 2))
    data = np.insert(data,0,values = np.ones(N),axis=1)
    #随机生成10个长度为3的数组
    point = np.random.uniform(-1,1,(2,2))
    k = (point[0][1] - point[1][1])/(point[0][0] - point[1][0])
    b = point[0][1] - k*point[0][0]
    WF = np.array([b,k,-1],dtype=np.float)
    #随机生成一条直线
    label =  np.int32(np.dot(data,WF)>0)
    label[label==0]=-1
    
    WPLA = np.zeros(3)
    return PLA(data,label,WPLA),WF


In [17]:
exp(10)

(array([ 0.        , -0.8235475 , -0.22121746]),
 array([ 0.00539658, -2.24907807, -1.        ]))

In [18]:
N=10
data = np.random.uniform(-1, 1, (N, 2))
data = np.insert(data,0,values = np.ones(N),axis=1)
#随机生成10个长度为3的数组
point = np.random.uniform(-1,1,(2,2))
k = (point[0][1] - point[1][1])/(point[0][0] - point[1][0])
b = point[0][1] - k*point[0][0]
WF = np.array([b,k,-1],dtype=np.float)

In [19]:
WF

array([-0.85308109,  0.08177847, -1.        ])

In [20]:
data = np.random.uniform(-1, 1, (N, 2))
data = np.insert(data,0,values = np.ones(N),axis=1)
#随机生成10个长度为3的数组
point = np.random.uniform(-1,1,(2,2))
k = (point[0][1] - point[1][1])/(point[0][0] - point[1][0])
b = point[0][1] - k*point[0][0]
WF = np.array([b,k,-1],dtype=np.float)
#随机生成一条直线
label =  np.int32(np.dot(data,WF)>0)
label[label==0]=-1

In [21]:
data

array([[ 1.        ,  0.6977079 ,  0.14061963],
       [ 1.        ,  0.02505007,  0.11774707],
       [ 1.        ,  0.65190357,  0.35875293],
       [ 1.        , -0.0742589 ,  0.90712994],
       [ 1.        ,  0.5970423 ,  0.71720045],
       [ 1.        , -0.31168016,  0.00894931],
       [ 1.        , -0.87059549, -0.66645882],
       [ 1.        , -0.18492123,  0.31585057],
       [ 1.        ,  0.6292421 , -0.16421078],
       [ 1.        , -0.19456269, -0.66183499]])